In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from simulai.models import Transformer
import joblib
from simulai.utilities import view_api # view_api uses the package torchview
import time
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
from torchviz import make_dot
from IPython.display import Image, display
import math
import copy
import json

In [2]:
# dataset_folder = "Battery_and_Heating_Data_in_Real_Driving_Cycles-Splits-of-TripB14"
# dataset_folder = "Battery_and_Heating_Data_in_Real_Driving_Cycles-Splits-in-Microtrips"
dataset_folder = "Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips"

In [3]:
NORMALIZE_DATA = True
BATCH_SIZE = 1024 * 40    # 1024 * 6
ML_MODEL_USES_CALCULATED_COLUMNS = True

In [4]:
required_columns = [
    "Time [s]", "SoC_0", "Q_rated", "Battery Voltage [V]", "Battery Current [A]", 
    # "Battery Temperature [°C]", 
    "Time_difference", "SoC [%]"
]    # always the target variable should be the last element of this list

calculated_columns = ["SoC_0", "Q_rated", "Time_difference"]

In [5]:
# get indexes of features that are relevant for the physics model
relevant_features_for_physics_model = ["SoC_0", "Battery Current [A]", "Time_difference", "Q_rated"]

feature_ids_relevant_for_physics_model = {}
for feature in relevant_features_for_physics_model:
    feature_ids_relevant_for_physics_model[feature] = required_columns.index(feature)

feature_ids_relevant_for_physics_model

{'SoC_0': 1, 'Battery Current [A]': 4, 'Time_difference': 5, 'Q_rated': 2}

# Load and prepare data

In [6]:
# Define which features will be used as input for ML model (original from dataset + calculated or only original from dataset)

# get indexes of calculated_columns, and 
all_columns_indexes = list(range(len(required_columns)-1))    # all indexes. The -1 is necessary to exclude the last column, the target variable.
indexed_calculated_columns = [idx for idx, column_name in enumerate(required_columns) if column_name in calculated_columns]
indexed_calculated_columns

# according to setting, decide if all columns will be used, or only the non-calculated ones
if ML_MODEL_USES_CALCULATED_COLUMNS: 
    features_indexes_for_ml = all_columns_indexes
else:
    features_indexes_for_ml = [i for i in all_columns_indexes if i not in indexed_calculated_columns]
    
features_indexes_for_ml

[0, 1, 2, 3, 4, 5]

## Load files as dataframes

In [7]:
def get_filepaths_with_extension(file_extension=".csv", directory="."):
    output = []
    
    for file in os.listdir(directory):  
        # Check if the file has the required extension
        if file.endswith(file_extension):
            output.append(f"{directory}/{file}") 

    return output

In [8]:
# get filepaths of all train micro-trips
train_trips_filepaths = get_filepaths_with_extension(file_extension=".csv", directory=f"./data/{dataset_folder}/train")
train_trips_filepaths[:5]

['./data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/train/TripA01.csv',
 './data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/train/TripA02.csv',
 './data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/train/TripA03.csv',
 './data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/train/TripA04.csv',
 './data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/train/TripA05.csv']

In [9]:
# get filepaths of all dev micro-trips
val_trips_filepaths = get_filepaths_with_extension(file_extension=".csv", directory=f"./data/{dataset_folder}/dev")
val_trips_filepaths[:5]

['./data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/dev/TripB19_part1.csv',
 './data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/dev/TripB20_part1.csv',
 './data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/dev/TripB21_part1.csv',
 './data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/dev/TripB22_part1.csv',
 './data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/dev/TripB23_part1.csv']

In [10]:
# get filepaths of all test micro-trips
test_trips_filepaths = get_filepaths_with_extension(file_extension=".csv", directory=f"./data/{dataset_folder}/test")
test_trips_filepaths[:5]

['./data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/test/TripB29_part1.csv',
 './data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/test/TripB30_part1.csv',
 './data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/test/TripB31_part1.csv',
 './data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/test/TripB32_part1.csv',
 './data/Battery_and_Heating_Data_in_Real_Driving_Cycles-Full_trips/test/TripB33.csv']

In [11]:
# load data

In [12]:
lst_train_dfs = [pd.read_csv(filepath, sep=";", encoding="ISO-8859-2") for filepath in train_trips_filepaths]
lst_train_dfs[0].head()

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,AirCon Power [kW],Heater Signal,Heater Voltage [V],Heater Current [A],Ambient Temperature [°C],Coolant Temperature Heatercore [°C],Requested Coolant Temperature [°C],Coolant Temperature Inlet [°C],Heat Exchanger Temperature [°C],Cabin Temperature Sensor [°C]
0,0.0,0.0,574.0,0.0,0.0,-0.03,0.0,391.4,-2.20,21.0,...,0.4,1,0,0,25.5,0,0,0,30.5,24.5
1,0.1,0.0,574.0,0.0,0.0,0.00,0.0,391.4,-2.21,21.0,...,0.4,1,0,0,25.5,0,0,0,30.5,24.5
2,0.2,0.0,574.0,0.0,0.0,-0.01,0.0,391.4,-2.26,21.0,...,0.4,1,0,0,25.5,0,0,0,30.5,24.5
3,0.3,0.0,574.0,0.0,0.0,-0.03,0.0,391.4,-2.30,21.0,...,0.4,1,0,0,25.5,0,0,0,30.5,24.5
4,0.4,0.0,574.0,0.0,0.0,-0.03,0.0,391.4,-2.30,21.0,...,0.4,1,0,0,25.5,0,0,0,30.5,24.5


In [13]:
lst_val_dfs = [pd.read_csv(filepath, sep=";", encoding="ISO-8859-2") for filepath in val_trips_filepaths]
lst_val_dfs[0].head()

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,Temperature Footweel Driver [°C],Temperature Footweel Co-Driver [°C],Temperature Feetvent Co-Driver [°C],Temperature Feetvent Driver [°C],Temperature Head Co-Driver [°C],Temperature Head Driver [°C],Temperature Vent right [°C],Temperature Vent central right [°C],Temperature Vent central left [°C],Temperature Vent right [°C]
0,0.0,0.0,512.0,0.0,0.0,-0.04,0.0,390.5,-9.70,6.0,...,4.33,3.72,4.06,4.06,4.59,6.07,3.19,3.10,3.10,3.19
1,0.1,0.0,512.0,0.0,0.0,-0.04,0.0,390.5,-9.70,6.0,...,4.33,3.72,4.06,4.07,4.61,6.09,3.19,3.10,3.11,3.21
2,0.2,0.0,512.0,0.0,0.0,-0.05,0.0,390.5,-9.70,6.0,...,4.34,3.72,4.06,4.08,4.64,6.12,3.21,3.11,3.12,3.22
3,0.3,0.0,512.0,0.0,0.0,-0.04,0.0,390.5,-10.05,6.0,...,4.35,3.73,4.06,4.09,4.66,6.15,3.23,3.12,3.14,3.24
4,0.4,0.0,512.0,0.0,0.0,-0.04,0.0,390.5,-10.40,6.0,...,4.36,3.74,4.06,4.10,4.69,6.17,3.24,3.13,3.16,3.26


In [14]:
lst_test_dfs = [pd.read_csv(filepath, sep=";", encoding="ISO-8859-2") for filepath in test_trips_filepaths]
lst_test_dfs[0].head()

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,Temperature Footweel Driver [°C],Temperature Footweel Co-Driver [°C],Temperature Feetvent Co-Driver [°C],Temperature Feetvent Driver [°C],Temperature Head Co-Driver [°C],Temperature Head Driver [°C],Temperature Vent right [°C],Temperature Vent central right [°C],Temperature Vent central left [°C],Temperature Vent right [°C]
0,0.0,0.0,472.0,0.0,0.0,-0.31,0.0,366.70,-11.40,11.0,...,10.79,13.14,24.14,23.62,10.17,10.87,17.16,15.85,12.18,16.55
1,0.1,0.0,472.0,0.0,0.0,-0.32,0.0,366.70,-11.40,11.0,...,10.79,13.14,24.14,23.62,10.17,10.87,17.16,15.85,12.18,16.55
2,0.2,0.0,472.0,0.0,0.0,-0.31,0.0,366.67,-11.84,11.0,...,10.79,13.14,24.14,23.62,10.17,10.87,17.16,15.85,12.18,16.55
3,0.3,0.0,472.0,0.0,0.0,-0.28,0.0,366.62,-12.54,11.0,...,10.79,13.14,24.14,23.62,10.17,10.87,17.16,15.85,12.18,16.55
4,0.4,0.0,472.0,0.0,0.0,-0.27,0.0,366.60,-12.71,11.0,...,10.79,13.14,24.14,23.62,10.17,10.87,17.16,15.85,12.18,16.55


## Add relevant calculated columns

In [15]:
def add_calculated_columns(df):
    battery_voltage=360
    battery_usable_capacity_kWh=18.8

    # Add initial SoC
    df["SoC_0"] = df["SoC [%]"][0]

    # Convert capacity to Ampere-seconds (As) using battery voltage
    Q_rated = (battery_usable_capacity_kWh * 1000) / battery_voltage * 3600   # Convert kWh to As

    # Add q_rated
    df["Q_rated"] = Q_rated

    # Time difference between samples
    time_difference = df['Time [s]'].diff().fillna(0)  # First diff is NaN, set to 0
    df["Time_difference"] = time_difference

    return df

In [16]:
lst_train_dfs = [add_calculated_columns(df) for df in lst_train_dfs]

In [17]:
lst_val_dfs = [add_calculated_columns(df) for df in lst_val_dfs]

In [18]:
lst_test_dfs = [add_calculated_columns(df) for df in lst_test_dfs]
lst_test_dfs[0]

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,Temperature Feetvent Driver [°C],Temperature Head Co-Driver [°C],Temperature Head Driver [°C],Temperature Vent right [°C],Temperature Vent central right [°C],Temperature Vent central left [°C],Temperature Vent right [°C],SoC_0,Q_rated,Time_difference
0,0.0,0.0,472.0,0.0,0.0,-0.31,0.0,366.70,-11.40,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.0
1,0.1,0.0,472.0,0.0,0.0,-0.32,0.0,366.70,-11.40,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.1
2,0.2,0.0,472.0,0.0,0.0,-0.31,0.0,366.67,-11.84,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.1
3,0.3,0.0,472.0,0.0,0.0,-0.28,0.0,366.62,-12.54,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.1
4,0.4,0.0,472.0,0.0,0.0,-0.27,0.0,366.60,-12.71,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9680,968.0,0.0,513.0,0.0,0.0,-0.04,0.0,344.40,-5.11,14.0,...,38.45,19.95,21.96,29.73,28.59,29.81,27.54,31.5,188000.0,0.1
9681,968.1,0.0,513.0,0.0,0.0,-0.04,0.0,344.40,-4.81,14.0,...,38.45,19.95,21.96,29.73,28.59,29.81,27.54,31.5,188000.0,0.1
9682,968.2,0.0,513.0,0.0,0.0,-0.04,0.0,344.40,-4.74,14.0,...,38.45,19.95,21.96,29.73,28.59,29.81,27.54,31.5,188000.0,0.1
9683,968.3,0.0,513.0,0.0,0.0,-0.07,0.0,344.40,-5.19,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.5,188000.0,0.1


## Restart Time column to 0 sec.

In [19]:
def restart_time_from_0(df):
    df["Time [s]"] = df["Time [s]"] - df["Time [s]"][0]
    return df

lst_train_dfs = [restart_time_from_0(df) for df in lst_train_dfs]
lst_val_dfs = [restart_time_from_0(df) for df in lst_val_dfs]
lst_test_dfs = [restart_time_from_0(df) for df in lst_test_dfs]

In [20]:
lst_train_dfs[0]

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,Heater Current [A],Ambient Temperature [°C],Coolant Temperature Heatercore [°C],Requested Coolant Temperature [°C],Coolant Temperature Inlet [°C],Heat Exchanger Temperature [°C],Cabin Temperature Sensor [°C],SoC_0,Q_rated,Time_difference
0,0.0,0.00,574.0,0.00,0.0,-0.03,0.0,391.40,-2.20,21.0,...,0,25.5,0,0,0,30.5,24.50,86.9,188000.0,0.0
1,0.1,0.00,574.0,0.00,0.0,0.00,0.0,391.40,-2.21,21.0,...,0,25.5,0,0,0,30.5,24.50,86.9,188000.0,0.1
2,0.2,0.00,574.0,0.00,0.0,-0.01,0.0,391.40,-2.26,21.0,...,0,25.5,0,0,0,30.5,24.50,86.9,188000.0,0.1
3,0.3,0.00,574.0,0.00,0.0,-0.03,0.0,391.40,-2.30,21.0,...,0,25.5,0,0,0,30.5,24.50,86.9,188000.0,0.1
4,0.4,0.00,574.0,0.00,0.0,-0.03,0.0,391.40,-2.30,21.0,...,0,25.5,0,0,0,30.5,24.50,86.9,188000.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10085,1008.5,20.71,565.0,25.06,-9.4,-0.28,0.0,387.91,-3.12,22.0,...,0,33.0,0,0,0,5.0,22.67,86.9,188000.0,0.1
10086,1008.6,20.60,565.0,23.57,-8.9,-0.24,0.0,387.96,-2.37,22.0,...,0,33.0,0,0,0,5.0,22.67,86.9,188000.0,0.1
10087,1008.7,20.44,565.0,22.55,-12.4,-0.31,0.0,388.01,-1.62,22.0,...,0,33.0,0,0,0,5.0,22.67,86.9,188000.0,0.1
10088,1008.8,20.30,565.0,22.55,-15.2,-0.40,0.0,388.06,-0.92,22.0,...,0,33.0,0,0,0,5.0,22.67,86.9,188000.0,0.1


In [21]:
lst_val_dfs[0]

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,Temperature Feetvent Driver [°C],Temperature Head Co-Driver [°C],Temperature Head Driver [°C],Temperature Vent right [°C],Temperature Vent central right [°C],Temperature Vent central left [°C],Temperature Vent right [°C],SoC_0,Q_rated,Time_difference
0,0.0,0.0,512.0,0.0,0.0,-0.04,0.0,390.5,-9.70,6.0,...,4.06,4.59,6.07,3.19,3.10,3.10,3.19,85.8,188000.0,0.0
1,0.1,0.0,512.0,0.0,0.0,-0.04,0.0,390.5,-9.70,6.0,...,4.07,4.61,6.09,3.19,3.10,3.11,3.21,85.8,188000.0,0.1
2,0.2,0.0,512.0,0.0,0.0,-0.05,0.0,390.5,-9.70,6.0,...,4.08,4.64,6.12,3.21,3.11,3.12,3.22,85.8,188000.0,0.1
3,0.3,0.0,512.0,0.0,0.0,-0.04,0.0,390.5,-10.05,6.0,...,4.09,4.66,6.15,3.23,3.12,3.14,3.24,85.8,188000.0,0.1
4,0.4,0.0,512.0,0.0,0.0,-0.04,0.0,390.5,-10.40,6.0,...,4.10,4.69,6.17,3.24,3.13,3.16,3.26,85.8,188000.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11905,1190.5,0.0,484.0,0.0,0.0,0.20,0.0,382.8,-0.70,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.8,188000.0,0.1
11906,1190.6,0.0,484.0,0.0,0.0,0.18,0.0,382.8,-0.70,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.8,188000.0,0.1
11907,1190.7,0.0,484.0,0.0,0.0,0.16,0.0,382.8,-0.67,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.8,188000.0,0.1
11908,1190.8,0.0,484.0,0.0,0.0,0.17,0.0,382.8,-0.62,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.8,188000.0,0.1


In [22]:
lst_test_dfs[0]

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,Temperature Feetvent Driver [°C],Temperature Head Co-Driver [°C],Temperature Head Driver [°C],Temperature Vent right [°C],Temperature Vent central right [°C],Temperature Vent central left [°C],Temperature Vent right [°C],SoC_0,Q_rated,Time_difference
0,0.0,0.0,472.0,0.0,0.0,-0.31,0.0,366.70,-11.40,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.0
1,0.1,0.0,472.0,0.0,0.0,-0.32,0.0,366.70,-11.40,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.1
2,0.2,0.0,472.0,0.0,0.0,-0.31,0.0,366.67,-11.84,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.1
3,0.3,0.0,472.0,0.0,0.0,-0.28,0.0,366.62,-12.54,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.1
4,0.4,0.0,472.0,0.0,0.0,-0.27,0.0,366.60,-12.71,11.0,...,23.62,10.17,10.87,17.16,15.85,12.18,16.55,31.5,188000.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9680,968.0,0.0,513.0,0.0,0.0,-0.04,0.0,344.40,-5.11,14.0,...,38.45,19.95,21.96,29.73,28.59,29.81,27.54,31.5,188000.0,0.1
9681,968.1,0.0,513.0,0.0,0.0,-0.04,0.0,344.40,-4.81,14.0,...,38.45,19.95,21.96,29.73,28.59,29.81,27.54,31.5,188000.0,0.1
9682,968.2,0.0,513.0,0.0,0.0,-0.04,0.0,344.40,-4.74,14.0,...,38.45,19.95,21.96,29.73,28.59,29.81,27.54,31.5,188000.0,0.1
9683,968.3,0.0,513.0,0.0,0.0,-0.07,0.0,344.40,-5.19,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.5,188000.0,0.1


## Remove irrelevant columns

In [23]:
def remove_irrelevant_columns(df):
    return df[required_columns]

In [24]:
lst_train_dfs = [remove_irrelevant_columns(df) for df in lst_train_dfs]
lst_train_dfs[0]

,Time [s],SoC_0,Q_rated,Battery Voltage [V],Battery Current [A],Time_difference,SoC [%]
0,0.0,86.9,188000.0,391.40,-2.20,0.0,86.9
1,0.1,86.9,188000.0,391.40,-2.21,0.1,86.9
2,0.2,86.9,188000.0,391.40,-2.26,0.1,86.9
3,0.3,86.9,188000.0,391.40,-2.30,0.1,86.9
4,0.4,86.9,188000.0,391.40,-2.30,0.1,86.9
...,...,...,...,...,...,...,...
10085,1008.5,86.9,188000.0,387.91,-3.12,0.1,81.5
10086,1008.6,86.9,188000.0,387.96,-2.37,0.1,81.5
10087,1008.7,86.9,188000.0,388.01,-1.62,0.1,81.5
10088,1008.8,86.9,188000.0,388.06,-0.92,0.1,81.5


In [25]:
lst_val_dfs = [remove_irrelevant_columns(df) for df in lst_val_dfs]
lst_val_dfs[0]

,Time [s],SoC_0,Q_rated,Battery Voltage [V],Battery Current [A],Time_difference,SoC [%]
0,0.0,85.8,188000.0,390.5,-9.70,0.0,85.8
1,0.1,85.8,188000.0,390.5,-9.70,0.1,85.8
2,0.2,85.8,188000.0,390.5,-9.70,0.1,85.8
3,0.3,85.8,188000.0,390.5,-10.05,0.1,85.8
4,0.4,85.8,188000.0,390.5,-10.40,0.1,85.8
...,...,...,...,...,...,...,...
11905,1190.5,85.8,188000.0,382.8,-0.70,0.1,71.6
11906,1190.6,85.8,188000.0,382.8,-0.70,0.1,71.6
11907,1190.7,85.8,188000.0,382.8,-0.67,0.1,71.6
11908,1190.8,85.8,188000.0,382.8,-0.62,0.1,71.6


In [26]:
lst_test_dfs = [remove_irrelevant_columns(df) for df in lst_test_dfs]
lst_test_dfs[0]

,Time [s],SoC_0,Q_rated,Battery Voltage [V],Battery Current [A],Time_difference,SoC [%]
0,0.0,31.5,188000.0,366.70,-11.40,0.0,31.5
1,0.1,31.5,188000.0,366.70,-11.40,0.1,31.5
2,0.2,31.5,188000.0,366.67,-11.84,0.1,31.5
3,0.3,31.5,188000.0,366.62,-12.54,0.1,31.5
4,0.4,31.5,188000.0,366.60,-12.71,0.1,31.5
...,...,...,...,...,...,...,...
9680,968.0,31.5,188000.0,344.40,-5.11,0.1,15.4
9681,968.1,31.5,188000.0,344.40,-4.81,0.1,15.4
9682,968.2,31.5,188000.0,344.40,-4.74,0.1,15.4
9683,968.3,31.5,188000.0,344.40,-5.19,0.1,15.4


## Prepare data
Help for data preparation from ChatGPT: https://chatgpt.com/share/678e1508-2250-800b-b898-922a65ba0061

In [27]:
# Detect device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [28]:
lst_train_dfs[0].columns

Index(['Time [s]', 'SoC_0', 'Q_rated', 'Battery Voltage [V]',
       'Battery Current [A]', 'Time_difference', 'SoC [%]'],
      dtype='object')

In [29]:
# separate X and y from dataframe
def separate_x_and_y(df):
    """
    It outputs two dataframes, the first one contains only the X values, the second only the y values.
    """
    
    df_X = df[required_columns[:-1]]
    df_y = df[["SoC [%]"]]
    return df_X, df_y

# separate x and y in dataframes of all sets
lst_train_data = [separate_x_and_y(df) for df in lst_train_dfs]
lst_val_data = [separate_x_and_y(df) for df in lst_val_dfs]
lst_test_dfs = [separate_x_and_y(df) for df in lst_test_dfs]

In [30]:
lst_train_data[0][0]

,Time [s],SoC_0,Q_rated,Battery Voltage [V],Battery Current [A],Time_difference
0,0.0,86.9,188000.0,391.40,-2.20,0.0
1,0.1,86.9,188000.0,391.40,-2.21,0.1
2,0.2,86.9,188000.0,391.40,-2.26,0.1
3,0.3,86.9,188000.0,391.40,-2.30,0.1
4,0.4,86.9,188000.0,391.40,-2.30,0.1
...,...,...,...,...,...,...
10085,1008.5,86.9,188000.0,387.91,-3.12,0.1
10086,1008.6,86.9,188000.0,387.96,-2.37,0.1
10087,1008.7,86.9,188000.0,388.01,-1.62,0.1
10088,1008.8,86.9,188000.0,388.06,-0.92,0.1


In [31]:
# save column names before parsing to numpy arrays
X_column_names = lst_train_data[0][0].columns
X_column_names

Index(['Time [s]', 'SoC_0', 'Q_rated', 'Battery Voltage [V]',
       'Battery Current [A]', 'Time_difference'],
      dtype='object')

In [32]:
# Parse data to numpy arrays
def parse_dataframes_to_numpy_arrays(dataframes):
    # id 0 contains dataframe with X values, id 1 contains dataframe with y values.
    return dataframes[0].to_numpy(), dataframes[1].to_numpy()

# parse dataframes of all sets
lst_train_data = [parse_dataframes_to_numpy_arrays(dataframes) for dataframes in lst_train_data]
lst_val_data = [parse_dataframes_to_numpy_arrays(dataframes) for dataframes in lst_val_data]
lst_test_dfs = [parse_dataframes_to_numpy_arrays(dataframes) for dataframes in lst_test_dfs]

In [33]:
lst_train_data[0][0][0:5]

array([[ 0.000e+00,  8.690e+01,  1.880e+05,  3.914e+02, -2.200e+00,
         0.000e+00],
       [ 1.000e-01,  8.690e+01,  1.880e+05,  3.914e+02, -2.210e+00,
         1.000e-01],
       [ 2.000e-01,  8.690e+01,  1.880e+05,  3.914e+02, -2.260e+00,
         1.000e-01],
       [ 3.000e-01,  8.690e+01,  1.880e+05,  3.914e+02, -2.300e+00,
         1.000e-01],
       [ 4.000e-01,  8.690e+01,  1.880e+05,  3.914e+02, -2.300e+00,
         1.000e-01]])

In [34]:
# Data scalling

# Initialize scalers for features and target
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Fit scalers on the training data only
train_features = [dataframes[0] for dataframes in lst_train_data]
train_target = [dataframes[1] for dataframes in lst_train_data]

# Combine training data to fit scalers
X_combined = np.vstack(train_features)
y_combined = np.vstack(train_target)
scaler_X.fit(X_combined)
scaler_y.fit(y_combined)

# Apply the scalers to each DataFrame

def normalize_data(X, y):
    X = scaler_X.transform(X)
    y = scaler_y.transform(y) 
    return X, y

if NORMALIZE_DATA:
    # Normalize training, validation, and test sets
    lst_train_data = [normalize_data(data[0], data[1]) for data in lst_train_data]
    lst_val_data = [normalize_data(data[0], data[1]) for data in lst_val_data]
    lst_test_dfs = [normalize_data(data[0], data[1]) for data in lst_test_dfs]

In [35]:
lst_train_data[0][0][0:5]

array([[-1.32002393e+00,  9.82556853e-01,  0.00000000e+00,
         1.24208573e+00,  3.67202277e-01, -1.28898875e+02],
       [-1.31989646e+00,  9.82556853e-01,  0.00000000e+00,
         1.24208573e+00,  3.66961693e-01,  7.75801960e-03],
       [-1.31976898e+00,  9.82556853e-01,  0.00000000e+00,
         1.24208573e+00,  3.65758777e-01,  7.75801960e-03],
       [-1.31964151e+00,  9.82556853e-01,  0.00000000e+00,
         1.24208573e+00,  3.64796444e-01,  7.75801960e-03],
       [-1.31951403e+00,  9.82556853e-01,  0.00000000e+00,
         1.24208573e+00,  3.64796444e-01,  7.75801960e-03]])

In [36]:
print(scaler_X.inverse_transform(lst_train_data[0][0][0:5]))

[[ 0.000e+00  8.690e+01  1.880e+05  3.914e+02 -2.200e+00  0.000e+00]
 [ 1.000e-01  8.690e+01  1.880e+05  3.914e+02 -2.210e+00  1.000e-01]
 [ 2.000e-01  8.690e+01  1.880e+05  3.914e+02 -2.260e+00  1.000e-01]
 [ 3.000e-01  8.690e+01  1.880e+05  3.914e+02 -2.300e+00  1.000e-01]
 [ 4.000e-01  8.690e+01  1.880e+05  3.914e+02 -2.300e+00  1.000e-01]]


In [37]:
def create_dataloader(list_datasets, batch_size=32):
    # function to convert data to DataLoader. Each dataloader corresponds to one time-series file. 
    # Since this function returns a list of dataloaders, it can be seen as returning a list of time-series files, whose data is split in batches.
    
    dataloaders = []
    for X_batch, y_batch in list_datasets:
        X_tensor = torch.tensor(X_batch, dtype=torch.float32).to(device)
        y_tensor = torch.tensor(y_batch, dtype=torch.float32).to(device)
        dataset = TensorDataset(X_tensor, y_tensor)
        dataloaders.append(DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=False))
    return dataloaders

# Create DataLoaders for each set
train_loaders = create_dataloader(lst_train_data,BATCH_SIZE)
val_loaders = create_dataloader(lst_val_data, BATCH_SIZE)
test_loaders = create_dataloader(lst_test_dfs, BATCH_SIZE)

In [38]:
# check if batch size is less or equal the BATCH_SIZE that has been set
for x, y in train_loaders[0]:
    print(f"Batch size from index 0 in dataloader: {len(x)}")
    break

print(f"BATCH_SIZE: {BATCH_SIZE}")

Batch size from index 0 in dataloader: 10090
BATCH_SIZE: 40960


In [39]:
"X values: ", lst_train_data[0][0], "Y values", lst_train_data[0][1]

('X values: ',
 array([[-1.32002393e+00,  9.82556853e-01,  0.00000000e+00,
          1.24208573e+00,  3.67202277e-01, -1.28898875e+02],
        [-1.31989646e+00,  9.82556853e-01,  0.00000000e+00,
          1.24208573e+00,  3.66961693e-01,  7.75801960e-03],
        [-1.31976898e+00,  9.82556853e-01,  0.00000000e+00,
          1.24208573e+00,  3.65758777e-01,  7.75801960e-03],
        ...,
        [-3.41897252e-02,  9.82556853e-01,  0.00000000e+00,
          9.19265151e-01,  3.81156105e-01,  7.75801963e-03],
        [-3.40622508e-02,  9.82556853e-01,  0.00000000e+00,
          9.24026517e-01,  3.97996933e-01,  7.75801949e-03],
        [-3.39347764e-02,  9.82556853e-01,  0.00000000e+00,
          9.27835609e-01,  4.12913094e-01,  7.75801963e-03]]),
 'Y values',
 array([[1.44532151],
        [1.44532151],
        [1.44532151],
        ...,
        [1.04412706],
        [1.04412706],
        [1.04412706]]))

# Create Model

In [40]:
# Transformer code based on https://chatgpt.com/share/6797a3c0-85b8-800b-92bd-45a43f1af080

In [41]:
# help from AI assistant https://chatgpt.com/share/6797bf62-3674-800b-84c6-3b455e40aa44
class PositionalEncoding(nn.Module):
    # based on https://stackoverflow.com/questions/77444485/using-positional-encoding-in-pytorch
    
    def __init__(self, d_model, max_len=5000):  # max_len can be increased if necessary
        super(PositionalEncoding, self).__init__()
        self.pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = self.pe.unsqueeze(0)  # Add batch dimension

    def forward(self, x):
        seq_len = x.size(1)  # Current sequence length
        if seq_len > self.pe.size(1):
            raise ValueError(f"Input sequence length {seq_len} exceeds max_len {self.pe.size(1)}.")
        return x + self.pe[:, :seq_len, :].to(x.device)
        

In [42]:
class CoulombCounting(nn.Module):
    # help from AI assistant: https://chatgpt.com/share/679931c6-a3b8-800b-870e-0a5eae035398

    def __init__(self):
        super(CoulombCounting, self).__init__()

    @staticmethod
    def calculate_soc(data):
        """
        Calculate the State of Charge (SoC) using the Coulomb Counting method.
        """
        
        # Extract columns data
        # TO DO: add logic to indicate which indexes contains which datafields. e.g.: SoC_0 has index 1, Battery_current has id 4, and so on.
        # print(f'battery_current: {data[:, feature_ids_relevant_for_physics_model["Battery Current [A]"]]}')
        soc_0 = data[:, feature_ids_relevant_for_physics_model["SoC_0"]]
        battery_current = data[:, feature_ids_relevant_for_physics_model["Battery Current [A]"]]
        time_difference = data[:, feature_ids_relevant_for_physics_model["Time_difference"]]
        q_rated = data[:, feature_ids_relevant_for_physics_model["Q_rated"]]
    
        # Coulomb Counting Method to estimate SOC
        estimated_soc = soc_0 + (battery_current * time_difference).cumsum() / q_rated * 100
        
        return estimated_soc

    def forward(self, x):
        return self.calculate_soc(x)

In [43]:
# test: get only data from columns that are not calculated
for train_loader in train_loaders:
    for X_batch, y_batch in train_loader:
        print(X_batch[:, features_indexes_for_ml])
        break
    break

tensor([[-1.3200e+00,  9.8256e-01,  0.0000e+00,  1.2421e+00,  3.6720e-01,
         -1.2890e+02],
        [-1.3199e+00,  9.8256e-01,  0.0000e+00,  1.2421e+00,  3.6696e-01,
          7.7580e-03],
        [-1.3198e+00,  9.8256e-01,  0.0000e+00,  1.2421e+00,  3.6576e-01,
          7.7580e-03],
        ...,
        [-3.4190e-02,  9.8256e-01,  0.0000e+00,  9.1927e-01,  3.8116e-01,
          7.7580e-03],
        [-3.4062e-02,  9.8256e-01,  0.0000e+00,  9.2403e-01,  3.9800e-01,
          7.7580e-03],
        [-3.3935e-02,  9.8256e-01,  0.0000e+00,  9.2784e-01,  4.1291e-01,
          7.7580e-03]], device='cuda:0')


In [44]:
# Create Physics Informed model

# Model parameters
d_model = 32    # 512 or 32 # Embedding dimension
num_heads = 2    # 8 or 2
num_layers = 2    # 6 or 2
dropout = 0.1
dim_feedforward = 2048    # 2048
input_dim = len(features_indexes_for_ml)    # according to the number of columns available for the ML model
output_dim = 1

# Define your embedding layer (assuming input features)
embedding_layer = nn.Linear(input_dim, d_model)

# Instantiate the Positional Encoding
positional_encoding = PositionalEncoding(d_model, max_len=BATCH_SIZE)

# Define Transformer Encoder layer
encoder_layer = nn.TransformerEncoderLayer(
    d_model=d_model, nhead=num_heads, dim_feedforward=dim_feedforward, dropout=dropout
)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

coulomb_counter = CoulombCounting()

class PhysicsformerModel(nn.Module):
    # help from AI assistant: https://chatgpt.com/share/679931c6-a3b8-800b-870e-0a5eae035398
    
    def __init__(self, use_physics=True, use_ml=True):
        super(PhysicsformerModel, self).__init__()
        self.use_physics = use_physics
        self.use_ml = use_ml

        if use_physics == False and use_ml==False:
            raise Exception ("At least one of these paramters should be true: use_physics, use_ml. They can not be False at the same time. ")

        # add physical layer, if required
        if self.use_physics:
            self.physics_layer = coulomb_counter

        # add ML layers, if required
        if self.use_ml:
            self.embedding = embedding_layer
            self.positional_encoding = positional_encoding
            self.transformer_encoder = transformer_encoder
            self.fc_out = nn.Linear(d_model, output_dim)
            

    def forward(self, x):
        x_out = None
        
        # Physical part
        if self.use_physics:
            x_physics = scaler_X.inverse_transform(x.cpu().numpy())    # undo scalling
            x_physics = self.physics_layer(x_physics)
            x_physics = x_physics.reshape(-1, 1)    # reshape it for the scaler
            x_physics = scaler_y.transform(x_physics)    # scale output from physics layer
            x_physics = torch.tensor(x_physics).unsqueeze(0)   # add extra dimension at the begining to match output format of ML model (see x_ml)
            x_physics = x_physics.to(device)    # parse it to pyTorch tensor

        # ML part
        if self.use_ml:
            # print(f"\tColumns available for ML: {len(x[1, features_indexes_for_ml])}")
            x_ml = self.embedding(x[:, features_indexes_for_ml])    # apply embedding
            x_ml = x_ml.unsqueeze(0)  # Add batch dimension. Solution from https://chatgpt.com/share/6798b16c-e9e8-800b-baed-379b5c4cf045
            x_ml = self.positional_encoding(x_ml)    # add positional encoding
            x_ml = self.transformer_encoder(x_ml)    # apply transformer enconder
            x_ml = self.fc_out(x_ml)

        # prepare model output
        if self.use_physics and self.use_ml:
            x_out = x_physics + x_ml    # sum physics output and ML output
        elif self.use_physics:
            x_out = x_physics    # output is only physics output
        elif self.use_ml:
            x_out = x_ml    # # output is only ML output
        
        return x_out

In [45]:
# Instantiate the model
use_physics = True
use_ml = True
model = PhysicsformerModel(use_physics=use_physics, use_ml=use_ml)
model

PhysicsformerModel(
  (physics_layer): CoulombCounting()
  (embedding): Linear(in_features=6, out_features=32, bias=True)
  (positional_encoding): PositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=32, bias=True)
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc_out): Linear(in_features=32, out_features=1, bias=True)
)

In [46]:
# change device of model
model = model.to(device)

In [47]:
# PyTorch optimizer
# learning_rate = 1e-3
learning_rate = 1e-4
if model.use_ml:
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [48]:
# Loss function
def rmse_loss(y_true, y_pred):
    mse_loss = torch.nn.functional.mse_loss(y_pred, y_true) 
    return torch.sqrt(mse_loss)  
    
# loss_function =  torch.nn.MSELoss()  
loss_function = rmse_loss 

# Training

In [49]:
# hyperparameters
# n_epochs = 150
n_epochs = 300
# patience = 10
patience = n_epochs    # this deactivates early stopping
evaluattion_frequency_in_epochs = 1

In [50]:
# count total number of train batches
n_train_batches = sum([len(data_loader) for data_loader in train_loaders])
n_train_batches

50

In [51]:
# count total number of dev batches
n_val_batches = sum([len(data_loader) for data_loader in val_loaders])
n_val_batches

10

In [52]:
# count total number of test batches
n_test_batches = sum([len(data_loader) for data_loader in test_loaders])
n_test_batches

10

In [53]:
# Training loop
best_val_loss = float('inf') 
best_model_state = copy.deepcopy(model.state_dict())
patience_counter = 0
best_model_epoch = 0

training_start_time = time.time()
all_training_losses = []
all_validation_losses = []

for epoch in range(n_epochs):
    epoch_loss = 0.0
    print(f"Epoch no.: {epoch}")

    # train if model contains ML
    if model.use_ml:
        model.train()
        for train_loader in train_loaders:
            for X_batch, y_batch in train_loader:
                y_batch = y_batch.unsqueeze(0)  # Add batch dimension, to match the dimensionality of the model output.
                
                # Forward pass
                predictions = model.forward(X_batch)
                loss = loss_function(predictions, y_batch)
        
                # Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        
                epoch_loss += loss.item()
    
        all_training_losses.append(epoch_loss/n_train_batches)
    else:
        print("Physical part only mode: No training performed.")

    # Validation
    if epoch % evaluattion_frequency_in_epochs == 0:
        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for val_loader in val_loaders:
                for X_batch, y_batch in val_loader:
                    y_batch = y_batch.unsqueeze(0)  # Add batch dimension, to match the dimensionality of the model output.
                    
                    predictions = model(X_batch)
                    val_loss += loss_function(predictions, y_batch).item()
                    
            avg_val_loss = val_loss / n_val_batches
            all_validation_losses.append(avg_val_loss)
            print(f"Training Loss: {epoch_loss/n_train_batches:.6f}, Validation Loss: {avg_val_loss:.6f}")

    # Early stopping verification
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0  
        best_model_state = copy.deepcopy(model.state_dict())
        # best_model_state = model.state_dict()
        best_model_epoch = epoch
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping")
            break    

training_end_time = time.time()

Epoch no.: 0
Training Loss: 0.222599, Validation Loss: 0.085952
Epoch no.: 1
Training Loss: 0.104941, Validation Loss: 0.061540
Epoch no.: 2
Training Loss: 0.091411, Validation Loss: 0.059376
Epoch no.: 3
Training Loss: 0.086600, Validation Loss: 0.056520
Epoch no.: 4
Training Loss: 0.083372, Validation Loss: 0.055362
Epoch no.: 5
Training Loss: 0.081068, Validation Loss: 0.055848
Epoch no.: 6
Training Loss: 0.079308, Validation Loss: 0.056174
Epoch no.: 7
Training Loss: 0.077759, Validation Loss: 0.056979
Epoch no.: 8
Training Loss: 0.076551, Validation Loss: 0.057873
Epoch no.: 9
Training Loss: 0.075440, Validation Loss: 0.058495
Epoch no.: 10
Training Loss: 0.074534, Validation Loss: 0.058590
Epoch no.: 11
Training Loss: 0.073464, Validation Loss: 0.058221
Epoch no.: 12
Training Loss: 0.072593, Validation Loss: 0.057548
Epoch no.: 13
Training Loss: 0.071447, Validation Loss: 0.057099
Epoch no.: 14
Training Loss: 0.070499, Validation Loss: 0.056710
Epoch no.: 15
Training Loss: 0.0695


KeyboardInterrupt



In [ ]:
# Calculate the training time in seconds
execution_time = training_end_time - training_start_time

# Convert execution time to hours and minutes
hours = int(execution_time // 3600)
minutes = int((execution_time % 3600) // 60)

print(f"Code execution time: {hours} hour{'s' if hours != 1 else ''}, {minutes} minute{'s' if minutes != 1 else ''}")

In [ ]:
print(f"Model saved was from epoch no. {best_model_epoch}. Best Validation loss: {best_val_loss}")

In [ ]:
# create model name to store it 
current_timestamp = datetime.today().strftime('%Y_%m_%d-%H-%M-%S')
model_name = f"""{'Physics' if model.use_physics else ''}{'-and-' if model.use_physics and model.use_ml else '-only-'}\
{'MachineLearning' if model.use_ml else ''}-timestamp_{current_timestamp}"""
model_name

# create folder to store results, if it does not exist yet
output_folder = f"results/{model_name}"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
len(all_training_losses)

In [ ]:
len(all_validation_losses)

In [ ]:
# Plot training and validation losses, if the evaluation frequency is 1
if evaluattion_frequency_in_epochs == 1:
    plt.figure(figsize=(10, 6))
    plt.plot(range(0, len(all_training_losses)), all_training_losses, label='Training Loss', linestyle='-')
    plt.plot(range(0, len(all_validation_losses)), all_validation_losses, label='Validation Loss', linestyle='--')
    
    plt.title('Training and Validation Loss Over Epochs', fontsize=16)
    plt.xlabel('Epochs', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(alpha=0.5)
    plt.xticks(range(0, len(all_training_losses), 10), rotation=45)  # display every 10th epoch in the xticks

    # Save the plot
    output_file = f"{output_folder}/plot-train_vs_evaluation_loss-{model_name}.png"
    plt.savefig(output_file, bbox_inches='tight', dpi=300)  # Save the plot with tight layout and high resolution
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Evaluate on test data (explicitly passing input data)
# load best model
model = None
model = PhysicsformerModel(use_physics=use_physics, use_ml=use_ml)
model.load_state_dict(best_model_state)  
model = model.to(device)

model.eval()

# get also the prediction for the data in each dataloader. Remember, each dataloader has batched data from one file of the dataset.
lst_all_loader_predictions = []    # each element of this list has predictions of an entire file of the dataset
test_loss = 0
with torch.no_grad():
    for test_loader in test_loaders:
        lst_loader_predictions = []    # each element of this list has predictions for one batch
        for X_batch, y_batch in test_loader:
                y_batch = y_batch.unsqueeze(0)  # Add batch dimension, to match the dimensionality of the model output.
            
                predictions = model(X_batch)
                test_loss += loss_function(predictions, y_batch).item()
                
                lst_loader_predictions.extend(predictions)    # save predictions
    
        # save predictions of the corresponding test_loader, since it has the prediction for one file of the dataset
        lst_all_loader_predictions.append(lst_loader_predictions)
        
avg_test_loss = test_loss / n_test_batches
print(f"Test Loss: {avg_test_loss:.6f}")

In [ ]:
lst_all_loader_predictions[0]

In [ ]:
# convert lst_all_loader_predictions into NumPy arrays
lst_all_loader_predictions = [
    t.cpu().detach().numpy() for sublist in lst_all_loader_predictions for t in sublist
]

# inverse_transform the scaling of the y_predicted values
lst_all_loader_predictions = [scaler_y.inverse_transform(predictions) for predictions in lst_all_loader_predictions]

# Flatten elements in lst_all_loader_predictions
# lst_all_loader_predictions = [arr.ravel() for arr in lst_all_loader_predictions]

In [ ]:
lst_all_loader_predictions[0]

In [ ]:
# save X_test, y_test and model predictions together in files

files_data = []    # each dataloader has batched data of 1 file of the (test) dataset, which will be stored here. So each element of this list is 1 file.
for idx_dataloader in range(len(test_loaders)):
    file_data = []    # each element is one batch of the file
    
    for X_batch, y_batch in test_loaders[idx_dataloader]:
        # combine X, y and  data to have everything in one list
        X_data = scaler_X.inverse_transform(X_batch.cpu().numpy())
        y_data = scaler_y.inverse_transform(y_batch.cpu().numpy())
        X_and_y = [list(l1) + list(l2) for l1, l2 in zip(X_data, y_data)]
    
        file_data.extend(X_and_y)

    file_data = np.array(file_data)
    file_predictions_array = np.array(lst_all_loader_predictions[idx_dataloader])
    file_data = [list(l1) + list(l2) for l1, l2 in zip(file_data, file_predictions_array)]
        
    files_data.append(file_data)

    # break

In [ ]:
files_data[0]

In [ ]:
# plot ground truth vs model Estimation of one test file

data = files_data[0]

# Extracting the last and second last elements of each sublist
last_elements = [row[-1] for row in data]
second_last_elements = [row[-2] for row in data]

# Plotting the data
plt.figure(figsize=(10, 6))
plt.plot(last_elements, label="Model estimation", linestyle='dashed')
plt.plot(second_last_elements, label="Ground Truth")

# Adding labels, title, and legend
plt.xlabel("Index")
plt.ylabel("Values")
plt.title("Ground Truth vs Estimation")
plt.legend()
plt.grid()

# Display the plot
plt.show()

## Store model and test results

In [ ]:
# store each element of files_data as csv file

# Create folder (if it doesn't exist) to store test predictions and X and y features
output_folder_test_files = f"{output_folder}/test_data_and_model_output"
os.makedirs(output_folder_test_files, exist_ok=True)

# create column name for model output
estimation_column_name = f"""{'Physics' if model.use_physics else ''}{'-and-' if model.use_physics and model.use_ml else '-only-'}\
{'MachineLearning' if model.use_ml else ''}"""

for idx_file_data in range(len(files_data)):
    file_data = files_data[idx_file_data]

    # create df
    columns = list(X_column_names.copy())

    columns.extend(["SoC [%]", estimation_column_name])
    # columns = ["Time [s]", "SoC_0", "Q_rated", "Battery Voltage [V]", "Battery Current [A]", "Time_difference", "SoC [%]", "Estimated SoC (Transformer model)"]
    df = pd.DataFrame(file_data, columns=columns)

    # save in created folder
    output_file = f"file-{idx_file_data}.csv"
    output_file = os.path.join(output_folder_test_files,output_file)
    df.to_csv(output_file, index=True, sep=";", encoding="ISO-8859-2")


In [ ]:
# Save the model
torch.save(best_model_state, f"{output_folder}/model-{model_name}.pth")

# Save scalers
joblib.dump(scaler_X, f"{output_folder}/X-scaler.pkl")
joblib.dump(scaler_y, f"{output_folder}/y-scaler.pkl")

In [ ]:
# export hyperparameters and other relevant variables
hyperparameters_and_other_relevant_variables = {
    "model_name": model_name,
    "NORMALIZE_DATA": NORMALIZE_DATA,
    "BATCH_SIZE": BATCH_SIZE,
    "ML_MODEL_USES_CALCULATED_COLUMNS": ML_MODEL_USES_CALCULATED_COLUMNS,
    "dataset_folder": dataset_folder,
    "required_columns": required_columns,
    "calculated_columns": calculated_columns,
    "device": torch.cuda.get_device_name(device),
    "d_model": d_model,
    "num_heads": num_heads,
    "num_layers": num_layers,
    "dropout": dropout,
    "dim_feedforward": dim_feedforward,
    "input_dim": input_dim,
    "output_dim": output_dim,
    "use_physics": use_physics,
    "use_ml": use_ml,
    "learning_rate": learning_rate,
    "n_epochs": n_epochs,
    "patience": patience,
    "evaluattion_frequency_in_epochs": evaluattion_frequency_in_epochs,
}


with open(f"{output_folder}/hyperparameters_and_relevant_variables.json", 'w') as file:
    json.dump(hyperparameters_and_other_relevant_variables, file)

In [ ]:
# export metrics
metrics = {
    "best validation loss": best_val_loss,
    "test set loss": avg_test_loss
}

with open(f"{output_folder}/metrics.json", 'w') as file:
    json.dump(metrics, file)

## Empty cuda cache if necessary

In [ ]:
if device.type == "cuda":
    torch.cuda.empty_cache()